# AIPerf -> LLM TCO Calculator Data Connector

This notebook shows you how to do LLM performance benchmarking with the NVIDIA AIPerf tool and then export the data to a TCO (Total Cost of Ownership) calculator in [Excel spreadsheet format](./LLM_TCO_Calculator.xlsx).


To execute this notebook, you can use the NVIDIA Triton server container:
```
docker run --gpus=all --ipc=host --net=host --rm -it -v $PWD:/myworkspace nvcr.io/nvidia/tritonserver:25.09-py3-sdk bash  
```

Then from within the docker interactive session:
```
pip install jupyterlab
jupyter lab --ip 0.0.0.0 --port=8888 --allow-root --notebook-dir=/myworkspace
```

First, we define some metadata fields describing the deployment environment.

**Notes:**
- NIM engine ID  provides both the backend type (e.g. TensorRT-LLM, vLLM or SGlang) and precision. You can find this information when the NIM container starts.

- This notebook collects data corresponding to a single deployment environment described by the metadata field. In this tutorial, we will make use of the `Meta-Llama-3-8B-Instruct` model. Note that NVIDIA NGC and HuggingFace model hub use slightly different identifier for this model.

In [2]:
meta_field = {
 'Model': "meta-llama/Meta-Llama-3-8B-Instruct",
 'GPU Type': "H100_80GB",
 'number_of_gpus': 1,
 'Precision': "BF16",
 'Execution Mode': "NIM-TRTLLM",
}

## Pre-requisite

First, we install the AIPerf tool in the Pytorch container. 
As a client-side LLM-focused benchmarking tool, NVIDIA AIPerf provides key metrics such as time to first token (TTFT), inter-token latency (ITL), tokens per second (TPS), requests per second (RPS) and more. AIPerf also supports any LLM inference service conforming to the OpenAI API specification, a widely accepted de facto standard in the industry. For this benchmarking guide, we’ll use NVIDIA NIM, a collection of inference microservices that offer high-throughput and low-latency inference for both base and fine-tuned LLMs. NIM features ease-of-use and enterprise-grade security and manageability. 

### Install AIPerf tool

In [1]:
%%bash
pip install aiperf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.7 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.4/753.4 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 855.9/855.9 kB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

### Setting up a NIM LLM server (optional)

If you don't already have a target for benchmarking, like an OpenAI compatible LLM service, let's setup one. 

NVIDIA NIM provides the easiest and quickest way to put LLMs and other AI foundation models into production. Read [A Simple Guide to Deploying Generative AI with NVIDIA NIM](https://developer.nvidia.com/blog/a-simple-guide-to-deploying-generative-ai-with-nvidia-nim/) or consult the latest [NIM LLM documentation](https://docs.nvidia.com/nim/large-language-models/latest/introduction.html) to get started, which will walk you through hardware requirements and prerequisites, including NVIDIA NGC API keys.

For convenience, the following commands have been provided for deploying NIM and executing inference from the [Getting Started Guide](https://docs.nvidia.com/nim/large-language-models/latest/getting-started.html):   

                                                                                                    
```
export NGC_API_KEY=<YOUR_NGC_API_KEY> 
export LOCAL_NIM_CACHE=~/.cache/nim

mkdir -p "$LOCAL_NIM_CACHE"

docker run -it --rm \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
    -u $(id -u) \
    -p 8000:8000 \
    nvcr.io/nim/meta/llama3-8b-instruct:latest
```


## Performance benchmarking script

The next step is to define the use cases (i.e. input/output sequence length scenarios) and carry out the benchmarking.

In [6]:
%%writefile benchmark.sh
#!/usr/bin/env bash

declare -A useCases

# Populate the array with use case descriptions and their specified input/output lengths
useCases["Translation"]="200/200"
useCases["Text classification"]="200/5"
useCases["Text summary"]="1000/200"

# Function to execute AIPerf with the input/output lengths as arguments
runBenchmark() {
   local description="$1"
   local lengths="${useCases[$description]}"
   IFS='/' read -r inputLength outputLength <<< "$lengths"

   echo "Running AIPerf for $description with input length $inputLength and output length $outputLength"
   #Runs
   for concurrency in 1 2 5 10 50 100 250; do

       local INPUT_SEQUENCE_LENGTH=$inputLength
       local INPUT_SEQUENCE_STD=0
       local OUTPUT_SEQUENCE_LENGTH=$outputLength
       local CONCURRENCY=$concurrency
       local REQUEST_COUNT=$(($CONCURRENCY * 3))
       local MODEL=meta/llama3-8b-instruct

       aiperf profile \
           -m $MODEL \
           --endpoint-type chat \
           --streaming \
           -u localhost:8000 \
           --synthetic-input-tokens-mean $INPUT_SEQUENCE_LENGTH \
           --synthetic-input-tokens-stddev $INPUT_SEQUENCE_STD \
           --concurrency $CONCURRENCY \
           --request-count $REQUEST_COUNT \
           --output-tokens-mean $OUTPUT_SEQUENCE_LENGTH \
           --extra-inputs min_tokens:$OUTPUT_SEQUENCE_LENGTH \
           --extra-inputs ignore_eos:true \
           --tokenizer meta-llama/Meta-Llama-3-8B-Instruct \
           --artifact-dir artifact/ISL${INPUT_SEQUENCE_LENGTH}_OSL${OUTPUT_SEQUENCE_LENGTH}/CON${CONCURRENCY}

   done
}

# Iterate over all defined use cases and run the benchmark script for each
for description in "${!useCases[@]}"; do
   runBenchmark "$description"
done


Overwriting benchmark.sh


This test will use the llama-3 tokenizer from HuggingFace, which is a guarded repository [https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct). You will need to apply for access, then login with your HF credential.

Open a terminal in you Jupyter lab interface, then login to HF:
```
   pip install huggingface_hub
   huggingface-cli login
```

Next, we execute the bash script, which will carry out the defined benchmarking scenarios and gather the data in a default directory named `artifacts` under the current working directory.

In [ ]:
%%bash
bash benchmark.sh

## Reading AIPerf data

Once performance benchmarking is done, we read and collect the results in a single data frame.

In [19]:
import os
import json
import pandas as pd

ISL_OSL_LIST = ["200_5", "200_200", "1000_200"]
CONCURRENCIES  = [1, 2, 5, 10, 50, 100, 250]
df = pd.DataFrame()

for concurrency in CONCURRENCIES :
    for isl_osl in ISL_OSL_LIST:
        ISL=isl_osl.split("_")[0]
        OSL=isl_osl.split("_")[1]
        
        with open(f'./artifact/ISL{ISL}_OSL{OSL}/CON{concurrency}/profile_export_aiperf.json', 'r') as f:
           data = json.load(f)
             
        row =  {
         'Inter Token 90th Percentile Latency (ms)': data["records"]["inter_token_latency"]["p90"],
         'Inter Token 99th Percentile Latency (ms)': data["records"]["inter_token_latency"]["p99"],
         'Inter Token Average Latency (ms)': data["records"]["inter_token_latency"]["avg"],
         'Time to First Token 90th Percentile Latency (ms)': data["records"]["ttft"]["p90"],
         'Time to First Token 99th Percentile Latency (ms)': data["records"]["ttft"]["p99"],
         'Time to First Token Average Latency (ms)': data["records"]["ttft"]["avg"],
         'Request 90th Percentile Latency (ms)': data["records"]["request_latency"]["p90"],
         'Request 99th Percentile Latency (ms)': data["records"]["request_latency"]["p99"],
         'Request Latency (ms)': data["records"]["request_latency"]["avg"],
         'Requests per Second': data["records"]["request_throughput"]["avg"],
         'Tokens per Second': data["records"]["output_token_throughput"]["avg"],
         'Seq Length (ISL/OSL)': isl_osl,
         'Concurrency': concurrency
        } 
        
        row = meta_field | row
        
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

## Exporting data to excel format

We next export the benchmarking data to a NIM TCO Calculator compatible format, which comprises both metadata fields as well as performance metric fields.

In [20]:
df.head()

,Model,GPU Type,number_of_gpus,Precision,Execution Mode,Inter Token 90th Percentile Latency (ms),Inter Token 99th Percentile Latency (ms),Inter Token Average Latency (ms),Time to First Token 90th Percentile Latency (ms),Time to First Token 99th Percentile Latency (ms),Time to First Token Average Latency (ms),Request 90th Percentile Latency (ms),Request 99th Percentile Latency (ms),Request Latency (ms),Requests per Second,Tokens per Second,Seq Length (ISL/OSL),Concurrency
0,meta-llama/Meta-Llama-3-8B-Instruct,H100_80GB,1,BF16,NIM-TRTLLM,4.949345,4.967092,4.867871,20.532956,20.988718,18.889364,39.753273,40.061448,38.360850,25.419948,127.099742,200_5,1
1,meta-llama/Meta-Llama-3-8B-Instruct,H100_80GB,1,BF16,NIM-TRTLLM,4.873248,4.874338,4.869679,20.860838,21.591705,18.887994,990.637183,991.584959,987.954030,1.010877,202.175430,200_200,1
2,meta-llama/Meta-Llama-3-8B-Instruct,H100_80GB,1,BF16,NIM-TRTLLM,4.708417,4.711556,4.700263,46.373498,46.790232,45.210619,981.959344,982.167204,980.562975,1.018414,203.682855,1000_200,1
3,meta-llama/Meta-Llama-3-8B-Instruct,H100_80GB,1,BF16,NIM-TRTLLM,6.257329,6.340666,5.838733,29.911478,33.787478,22.652078,53.725809,57.137345,46.007011,41.722412,208.612062,200_5,2
4,meta-llama/Meta-Llama-3-8B-Instruct,H100_80GB,1,BF16,NIM-TRTLLM,4.920428,4.921603,4.912614,29.580081,33.868314,22.379320,1007.499842,1011.103446,999.989505,1.995674,399.134707,200_200,2


In [21]:
!pip install openpyxl

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/texttable-1.7.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/igraph-0.11.8-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce thi

In [22]:
columns = [
 'Model',
 'GPU Type',
 'Seq Length (ISL/OSL)',
 'number_of_gpus',
 'Concurrency',
 'Precision',
 'Execution Mode',
 'Inter Token 90th Percentile Latency (ms)',
 'Inter Token 99th Percentile Latency (ms)',
 'Inter Token Average Latency (ms)',
 'Time to First Token 90th Percentile Latency (ms)',
 'Time to First Token 99th Percentile Latency (ms)',
 'Time to First Token Average Latency (ms)',
 'Request 90th Percentile Latency (ms)',
 'Request 99th Percentile Latency (ms)',
 'Request Latency (ms)',
 'Requests per Second',
 'Tokens per Second'
 ]
df[columns].to_excel('data.xlsx', index=False)


## Importing the data to the TCO calculator

The [NIM TCO calculator tool](LLM_TCO_Calculator.xlsx) is implemented as an Excel spreadsheet. You can use MS Excel spreadsheet to open the excel file above, then simply copy the data rows into the "data" subsheet of the TCO calculator. That will complete the import phase and make the new data available in the TCO calculator.